$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
$\newcommand{\gf}{\texttt{GiRaFFEFood}}$
## $\gf$: An Einstein Toolkit Initial Data Thorn for $\giraffe$

### NRPy+ Source Code for this module: [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py) , which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFEFood_HO.ipynb) on using NRPy+ to construct these Exact Wald initial data as SymPy expressions.


In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up *initial data* for $\giraffe$. In a [previous tutorial module](Tutorial-MaxwellCartesian.ipynb), we used NRPy+ to contruct the SymPy expressions for plane-wave initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

### Step 1: Call on NRPy+ to convert the SymPy expression for the Eact Wald initial data into a C-code kernel. 

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py), and we simply import them for use here.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1c: Within the ETK, the 3D gridfunctions x, y, and z store the
#          Cartesian grid coordinates. Setting the gri.xx[] arrays
#          to point to these gridfunctions forces NRPy+ to treat
#          the Cartesian coordinate gridfunctions properly --
#          reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

import GiRaFFEFood_HO as gfho
gfho.GiRaFFEFood_HO()

# Step 2: Create the C code output kernel.
GiRaFFEFood_A_v_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfho.AD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfho.AD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfho.AD[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfho.ValenciavU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfho.ValenciavU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfho.ValenciavU[2]),\
                            ]

GiRaFFEFood_S_B_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=gfho.BU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=gfho.BU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=gfho.BU[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","StildeD0"),rhs=gfho.StildeD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","StildeD1"),rhs=gfho.StildeD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","StildeD2"),rhs=gfho.StildeD[2]),\
                            ]

GiRaFFEFood_A_v_CKernel = fin.FD_outputC("returnstring",GiRaFFEFood_A_v_to_print)
GiRaFFEFood_S_B_CKernel = fin.FD_outputC("returnstring",GiRaFFEFood_S_B_to_print)
GiRaFFEFood_S_B_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + GiRaFFEFood_S_B_CKernel

GiRaFFEFood_A_v_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFEFood_A_v_CKernel.replace("time","cctk_time"))
GiRaFFEFood_S_B_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFEFood_S_B_CKernel.replace("time","cctk_time"))

# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFEFood     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFEFood/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFEFood/src/GiRaFFEFood_A_v_ExactWald.h", "w") as file:
    file.write(str(GiRaFFEFood_A_v_looped))

with open("GiRaFFEFood/src/GiRaFFEFood_S_B_ExactWald.h", "w") as file:
    file.write(str(GiRaFFEFood_S_B_looped))

# Step 5: Import the function to calculate u0 and write it to a file.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0etc
#u0etc.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)

with open("GiRaFFEFood/src/computeu0_Cfunction.h", "w") as file:
    file.write(u0etc.computeu0_Cfunction)


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

We will write another C file with the functions we need here.

In [2]:
%%writefile GiRaFFEFood/src/InitialData.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
void GiRaFFE_set_A_v(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                     const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,const CCTK_REAL *u0GF,
                     CCTK_REAL *AD0GF,CCTK_REAL *AD1GF,CCTK_REAL *AD2GF,
                     CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "GiRaFFEFood_A_v_ExactWald.h"

}

void GiRaFFE_set_S_B(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                     const CCTK_REAL *ValenciavU0GF,const CCTK_REAL *ValenciavU1GF,const CCTK_REAL *ValenciavU2GF,const CCTK_REAL *u0GF,
                     const CCTK_REAL *alphaGF,const CCTK_REAL *betaU0GF,const CCTK_REAL *betaU1GF,const CCTK_REAL *betaU2GF,
                     const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                     CCTK_REAL *BU0GF,CCTK_REAL *BU1GF,CCTK_REAL *BU2GF,
                     CCTK_REAL *StildeD0GF,CCTK_REAL *StildeD1GF,CCTK_REAL *StildeD2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "GiRaFFEFood_S_B_ExactWald.h"

}

void Write_to_HydroBase(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                        const CCTK_REAL *ValenciavU0,const CCTK_REAL *ValenciavU1,const CCTK_REAL *ValenciavU2,
                        const CCTK_REAL *AD0,const CCTK_REAL *AD1,const CCTK_REAL *AD2,
                        const CCTK_REAL *BU0,const CCTK_REAL *BU1,const CCTK_REAL *BU2,
                        CCTK_REAL *vel, CCTK_REAL *Avec,CCTK_REAL *Bvec) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT idx3;
  CCTK_INT idx4[3];
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Bvec[idx4[0]] = BU0[idx3];
              Bvec[idx4[1]] = BU1[idx3];
              Bvec[idx4[2]] = BU2[idx3];
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID thorns.
          }
      }
  }
}

void find_u0(CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF,CCTK_REAL alpha,
             const CCTK_REAL gammaDD00,const CCTK_REAL gammaDD01,const CCTK_REAL gammaDD02,const CCTK_REAL gammaDD11,const CCTK_REAL gammaDD12,const CCTK_REAL gammaDD22,
             CCTK_REAL *u0GF,const CCTK_INT idx)
{
  DECLARE_CCTK_PARAMETERS;
  CCTK_REAL u0;
  CCTK_REAL ValenciavU0 = ValenciavU0GF[idx];
  CCTK_REAL ValenciavU1 = ValenciavU1GF[idx];
  CCTK_REAL ValenciavU2 = ValenciavU2GF[idx];

#include "computeu0_Cfunction.h"

  u0GF[idx] = u0;
  ValenciavU0GF[idx] = ValenciavU0;
  ValenciavU1GF[idx] = ValenciavU1;
  ValenciavU2GF[idx] = ValenciavU2;
}

void GiRaFFE_ExactWaldID(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  GiRaFFE_set_A_v(cctkGH,cctk_lsh,cctk_nghostzones,
                  x,y,z,u0,
                  AD0,AD1,AD2,
                  ValenciavU0,ValenciavU1,ValenciavU2);

  #pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              const CCTK_INT idx = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              find_u0(ValenciavU0,ValenciavU1,ValenciavU2,alp[idx],
                      gxx[idx],gxy[idx],gxz[idx],gyy[idx],gyz[idx],gzz[idx],
                      u0,idx);
          }
      }
  }
  
  GiRaFFE_set_S_B(cctkGH,cctk_lsh,cctk_nghostzones,
                  ValenciavU0,ValenciavU1,ValenciavU2,u0,
                  alp,betax,betay,betaz,
                  gxx,gxy,gxz,gyy,gyz,gzz,
                  BU0,BU1,BU2,
                  StildeD0,StildeD1,StildeD2);
  
  Write_to_HydroBase(cctkGH,cctk_lsh,cctk_nghostzones,
                     ValenciavU0,ValenciavU1,ValenciavU2,
                     AD0,AD1,AD2,
                     BU0,BU1,BU2,
                     vel,Avec,Bvec);
}

Overwriting GiRaFFEFood/src/InitialData.c


### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

In [3]:
%%writefile GiRaFFEFood/interface.ccl
implements: GiRaFFEFood_HO
inherits: admbase GiRaFFE_HO grid HydroBase


Overwriting GiRaFFEFood/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [4]:
%%writefile GiRaFFEFood/param.ccl
shares: grid

USES KEYWORD type

shares: GiRaFFE_HO
USES CCTK_REAL GAMMA_SPEED_LIMIT

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "ExactWald"      :: "Exact Wald initial data"
} "ExactWald"

restricted:
CCTK_REAL M "Kerr-Schild BH mass. Probably should always set M=1."
{
  0.0:* :: "Must be positive"
} 1.0


Overwriting GiRaFFEFood/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [5]:
%%writefile GiRaFFEFood/schedule.ccl
schedule GiRaFFE_ExactWaldID at CCTK_INITIAL as GiRaFFEFood
{
  STORAGE: GiRaFFE_HO::GiRaFFE_vars[3]
  STORAGE: GiRaFFE_HO::GiRaFFE_Vs[1]
  STORAGE: GiRaFFE_HO::GiRaFFE_Bs[1]
  STORAGE: GiRaFFE_HO::GiRaFFE_u0[1]
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
  WRITES: GiRaFFE_HO::AD0(Everywhere)
  WRITES: GiRaFFE_HO::AD1(Everywhere)
  WRITES: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
  WRITES: GiRaFFE_HO::StildeD0(Everywhere)
  WRITES: GiRaFFE_HO::StildeD1(Everywhere)
  WRITES: GiRaFFE_HO::StildeD2(Everywhere)
} "Initial data for GiRaFFE"


Overwriting GiRaFFEFood/schedule.ccl


#### Step 2c: Add the C code to the Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile GiRaFFEFood/src/make.code.defn
SRCS = InitialData.c

Overwriting GiRaFFEFood/src/make.code.defn
